<center><h2>ALTeGraD 2021<br>Lab Session 1: HAN</h2><h3>Hierarchical Attention Network Using GRU</h3> 09 / 11 / 2021<br> M. Kamal Eddine, H. Abdine</center>




In [6]:
# In case you are using google colab:
# uncomment the following two lines: 

%tensorflow_version 1.9
!pip install keras==2.2.5

!wget -c "https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%2199289&authkey=AHgxt3xmgG0Fu5A" -O "data.zip"
!unzip data.zip

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.9`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
--2021-11-16 11:20:07--  https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%2199289&authkey=AHgxt3xmgG0Fu5A
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://vqtlqw.am.files.1drv.com/y4m69Z6VlIPw6VRgDKNtwhVyqWvt-P5PmnBMJLvv5NhXw5GbH3bKMVqQzmgFqZkj2IVjvJEIJKME3EAXhNGI7TuxDLTeHpq6nS1UwyMNgMQ7WGJPo6ISUqQx7tFxdWHSNSAoVuQj-F55tlTs6fV2p2rqToB8h3k-GrlD230mniFEBFEcnE260gCXgB5XAkdtQQTveS2UH4AlrlA8VGC0Z8ceA/data.zip?download&psid=1 [following]
--2021-11-16 11:20:08--  https://vqtlqw.am.files.1drv.com/y4m69Z6VlIPw6VRgDKNtwhVyqWvt-P5PmnBMJLvv5NhXw5GbH3bKMVqQzmgFqZkj2IVjvJEIJKME3EAXhNGI7TuxDLTeHpq6nS1UwyMNgMQ7WGJP

# = = = = = Attention Layer = = = = =

In [7]:
def dot_product(x, kernel):
    """
    https://github.com/richliao/textClassifier/issues/13#issuecomment-377323318
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [8]:
import keras.backend as K
from keras.layers import Layer as Layer
from keras import initializers, regularizers, constraints

class AttentionWithContext(Layer):
    """
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    
    def __init__(self, return_coefficients=None,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform')
        
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    
    def build(self, input_shape):
        assert len(input_shape) == 3
        
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        
        super(AttentionWithContext, self).build(input_shape)
    
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        
        if self.bias:
            uit += self.b
        
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        
        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        
        a = K.expand_dims(a)
        weighted_input = K.sum(x*a, axis = 1)
        ### fill the gap ### # compute the attentional vector
        
        if self.return_coefficients:
            return [weighted_input,a]  ### fill the gap - [attentional vector, coefficients] ###
        else:
            return weighted_input ### fill the gap - attentional vector only ###
    
    
    
    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

# = = = = = Bidirectional GRU = = = = =
#### fill the gaps in the bidir_gru function below ###
#### add a RNN-GRU layer and a bidirectional wrapper ###
#### bidirectional: search for 'bidirectional' [here](https://keras.io/layers/wrappers/)
#### GRU: search for 'GRU' [here](https://keras.io/layers/recurrent/)
#### layers can be combined by nesting them as: layer_b(parameters_b)(layer_a(parameters_a)(input))

In [9]:
from keras.layers import Bidirectional, GRU

def bidir_gru(my_seq,n_units):
    #gru = GRU(n_units,return_sequences =True)
    #bidir = Bidirectional(gru)
    '''
    just a convenient wrapper for bidirectional RNN with GRU units
    '''
    #return bidir(my_seq)
    return Bidirectional(GRU(n_units,return_sequences =True))(my_seq)
    #return GRU(n_units,return_sequences =True)(my_seq)

# = = = = = Parameters = = = = =

In [10]:
import sys
import json
import operator
import numpy as np

#path_root = 'for_moodle/'
path_root = %pwd
path_to_data = path_root + '/data/'

d = 30 # dimensionality of word embeddings
n_units = 50 # RNN layer dimensionality
drop_rate = 0.5 # dropout
mfw_idx = 2 # index of the most frequent words in the dictionary 
            # 0 is for the special padding token
            # 1 is for the special out-of-vocabulary token

padding_idx = 0
oov_idx = 1
batch_size = 32
nb_epochs = 6
my_optimizer = 'adam'
my_patience = 2 # for early stopping strategy

# = = = = = Data Loading = = = = =

In [11]:
my_docs_array_train = np.load(path_to_data + 'docs_train.npy')
my_docs_array_test = np.load(path_to_data + 'docs_test.npy')

my_labels_array_train = np.load(path_to_data + 'labels_train.npy')
my_labels_array_test = np.load(path_to_data + 'labels_test.npy')

# load dictionary of word indexes (sorted by decreasing frequency across the corpus)
with open(path_to_data + 'word_to_index.json', 'r') as my_file:
    word_to_index = json.load(my_file)

# invert mapping
index_to_word = dict((v,k) for k,v in word_to_index.items()) ### fill the gap (use a dict comprehension) ###

# = = = = = Defining Architecture = = = = =

In [12]:
from keras.models import Model
from keras.layers import Input, Embedding, Dropout, TimeDistributed, Dense

sent_ints = Input(shape=(my_docs_array_train.shape[2],)) # vec of ints of variable size

sent_wv = Embedding(input_dim=len(index_to_word)+2, # vocab size
                    output_dim=d, # dimensionality of embedding space
                    input_length=my_docs_array_train.shape[2],
                    trainable=True
                    )(sent_ints)

sent_wv_dr = Dropout(drop_rate)(sent_wv)
sent_wa = bidir_gru(sent_wv_dr,n_units)
### fill the gap ### # get the annotations for each word in the sent
sent_att_vec,word_att_coeffs = AttentionWithContext(return_coefficients = True)(sent_wa)
### fill the gap ### # get the attentional vector for the sentence
sent_att_vec_dr = Dropout(drop_rate)(sent_att_vec)                      
sent_encoder = Model(sent_ints,sent_att_vec_dr)

doc_ints = Input(shape=(my_docs_array_train.shape[1],my_docs_array_train.shape[2],))
sent_att_vecs_dr = TimeDistributed(sent_encoder)(doc_ints)
### fill the gap ### # apply the sentence encoder model to each sentence in the document. Search for 'TimeDistributed' in https://keras.io/layers/wrappers/
doc_sa = bidir_gru(sent_att_vecs_dr,n_units) ### fill the gap ### # get annotations for each sent in the doc
doc_att_vec,sent_att_coeffs = AttentionWithContext(return_coefficients =True)(doc_sa)
### fill the gap ### # get attentional vector for the doc
doc_att_vec_dr = Dropout(drop_rate)(doc_att_vec)
                  
preds = Dense(units=1,
              activation='sigmoid')(doc_att_vec_dr)
model = Model(doc_ints,preds)

model.compile(loss='binary_crossentropy',
              optimizer = my_optimizer,
              metrics = ['accuracy'])

print('model compiled')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
model compiled


# = = = = = Training = = = = =

In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

loading_pretrained = False

if not loading_pretrained:
    early_stopping = EarlyStopping(monitor='val_acc',
                                   patience=my_patience,
                                   mode='max')
    
    # save model corresponding to best epoch
    checkpointer = ModelCheckpoint(filepath=path_to_data + 'model', 
                                   verbose=1, 
                                   save_best_only=True,
                                   save_weights_only=True)
    
    # 200s/epoch on CPU - reaches 84.38% accuracy in 2 epochs
    model.fit(my_docs_array_train,my_labels_array_train, validation_data = (my_docs_array_test, my_labels_array_test), epochs=nb_epochs, batch_size = batch_size, callbacks = [early_stopping,checkpointer] )
    ### fill the gap ### # call the .fit() method on your model with the arguments: my_docs_array_train, my_labels_array_train, batch_size, nb_epochs, my_docs_array_test, my_labels_array_test, early_stopping
    # look at: https://keras.io/models/sequential/#fit

else:
    model.load_weights(path_to_data + 'model')




Train on 25000 samples, validate on 25000 samples
Epoch 1/6





25000/25000 [==============================] - 123s 5ms/step - loss: 0.4576 - acc: 0.7692 - val_loss: 0.3701 - val_acc: 0.8394

Epoch 00001: val_loss improved from inf to 0.37009, saving model to /content/data/model
Epoch 2/6
25000/25000 [==============================] - 119s 5ms/step - loss: 0.2630 - acc: 0.8946 - val_loss: 0.3814 - val_acc: 0.8383

Epoch 00002: val_loss did not improve from 0.37009
Epoch 3/6
25000/25000 [==============================] - 119s 5ms/step - loss: 0.1866 - acc: 0.9253 - val_loss: 0.4409 - val_acc: 0.8202

Epoch 00003: val_loss did not improve from 0.37009


# = = = = = Extraction of Attention Coefficients = = = = =

In [14]:
from keras.models import Model

# define intermediate models
### fill the two gaps below ###
get_word_att_coeffs = Model(sent_ints, word_att_coeffs) # attention coeffs over the words in a sent
get_sent_attention_coeffs = Model(doc_ints, sent_att_coeffs) # attention coeffs over the sents in the doc

my_review = my_docs_array_test[-1:,:,:] # select last review
# convert integer review to text
index_to_word[1] = 'OOV'
my_review_text = [[index_to_word[idx] for idx in sent if idx in index_to_word] for sent in my_review.tolist()[0]]

# = = = = = Attention Over Sentences in the Document = = = = =

In [15]:
sent_coeffs = get_sent_attention_coeffs.predict(my_review)
sent_coeffs = sent_coeffs[0,:,:]

for elt in zip(sent_coeffs[:,0].tolist(),[' '.join(elt) for elt in my_review_text]):
    print(round(elt[0]*100,2),elt[1])

13.26 There 's a sign on The Lost Highway that says : OOV SPOILERS OOV ( but you already knew that , did n't you ? )
25.44 Since there 's a great deal of people that apparently did not get the point of this movie , I 'd like to contribute my interpretation of why the plot
15.08 As others have pointed out , one single viewing of this movie is not sufficient .
22.31 If you have the DVD of MD , you can OOV ' by looking at David Lynch 's 'Top 10 OOV to OOV MD ' ( but only upon second
11.99 ; ) First of all , Mulholland Drive is downright brilliant .
6.63 A masterpiece .
5.29 This is the kind of movie that refuse to leave your head .


# = = = = = Attention Over Words in Each Sentence = = = = =

In [16]:
from keras.backend.tensorflow_backend import _to_tensor

my_review_tensor = _to_tensor(my_review,dtype='float32') # a layer, unlike a model, requires a TensorFlow tensor as input

word_coeffs = TimeDistributed(get_word_att_coeffs)(my_review_tensor) ### fill the gap ### # get the word attentional coefficients for each sentence in the document
word_coeffs = K.eval(word_coeffs) # shape = (1, 7, 30, 1): (batch size, nb of sents in doc, nb of words per sent, coeff)
word_coeffs = word_coeffs[0,:,:,0] # shape = (7, 30) (coeff for each word in each sentence)
word_coeffs = sent_coeffs * word_coeffs # re-weight by sentence importance
word_coeffs = np.round((word_coeffs*100).astype(np.float64),2)

word_coeffs_list = word_coeffs.tolist()

# match text and coefficients
text_word_coeffs = [list(zip(words,word_coeffs_list[idx][:len(words)])) for idx,words in enumerate(my_review_text)]

for sent in text_word_coeffs:
    [print(elt) for elt in sent]  
    print('= = = =')

# sort words by importance within each sentence
text_word_coeffs_sorted = [sorted(elt,key=operator.itemgetter(1),reverse=True) for elt in text_word_coeffs]

for sent in text_word_coeffs_sorted:
    [print(elt) for elt in sent]
    print('= = = =')

('There', 0.32)
("'s", 0.36)
('a', 0.66)
('sign', 0.16)
('on', 0.22)
('The', 0.19)
('Lost', 3.16)
('Highway', 1.86)
('that', 0.72)
('says', 0.15)
(':', 0.21)
('OOV', 0.18)
('SPOILERS', 0.1)
('OOV', 0.11)
('(', 0.09)
('but', 0.49)
('you', 0.84)
('already', 0.71)
('knew', 0.84)
('that', 0.37)
(',', 0.34)
('did', 0.12)
("n't", 0.08)
('you', 0.13)
('?', 0.11)
(')', 0.41)
= = = =
('Since', 0.24)
('there', 0.14)
("'s", 0.18)
('a', 0.51)
('great', 6.94)
('deal', 8.25)
('of', 3.98)
('people', 2.9)
('that', 1.02)
('apparently', 0.39)
('did', 0.16)
('not', 0.04)
('get', 0.04)
('the', 0.04)
('point', 0.01)
('of', 0.02)
('this', 0.04)
('movie', 0.11)
(',', 0.09)
('I', 0.09)
("'d", 0.02)
('like', 0.02)
('to', 0.02)
('contribute', 0.01)
('my', 0.03)
('interpretation', 0.04)
('of', 0.02)
('why', 0.03)
('the', 0.03)
('plot', 0.02)
= = = =
('As', 1.68)
('others', 1.59)
('have', 0.55)
('pointed', 0.16)
('out', 0.17)
(',', 0.22)
('one', 0.27)
('single', 0.36)
('viewing', 0.86)
('of', 0.46)
('this', 0.85)